Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. Each borrower fills out a comprehensive application, providing his past financial history, the reason for the loan, etc. Each borrower's credit score is then evaluated using past historical data and an interest rate is assinged to the borrower. In this task, we'll focus on approved loans data from 2007 to 2011.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
loans_2007 = pd.read_csv('loans_2007.csv')
print(loans_2007.head())

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade    ...    last_pymnt_amnt  \
0   10.65%       162.87     B        B2    ...             171.62   
1   15.27%        59.83     C        C4    ...             119.66   
2   15.96%        84.33     C        C5    ...             649.91   
3   13.49%       339.31     C        C1    ...             357.48   
4   12.69%        67.79     B        B5    ...              67.79   

  last_credit_pull_d collections_12_mths_ex_med  policy_code application_type  \
0           Jun-2016               

C:\Users\chaouki\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The Dataframe contains many columns and can be cumbersome to try to explore all at once. After careful study of each column, the following columns need to be dropped to avoid data leakage leading to overfitting:
-  id: randomly generated field by Lending Club for unique identification purposes only.
-  member_id: also a randomly generated field by Lending Club for unique identification purposes only.
-  funded_amnt: leaks data from the future (after the loan is already started to be funded).
-  funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded).
-  grade: contains redundant information as the interest rate column (int_rate).
-  sub_grade: also contains redundant information as the interest rate column (int_rate).
-  emp_title: requires other data and a lot of processing to potentially be useful.
-  issue_d: leaks data from the future (after the loan is already completed funded).
-  zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in).
-  out_prncp: leaks data from the future, (after the loan already started to be paid off).
-  out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off).
-  total_pymnt: also leaks data from the future, (after the loan already started to be paid off).
-  total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off).
-  total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off).
-  total_rec_int: leaks data from the future, (after the loan already started to be paid off),
-  total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
-  recoveries: also leaks data from the future, (after the loan already started to be paid off),
-  collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
-  last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
-  last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

In [2]:
# dropping the columns from the dataset
loans_2007.drop(columns = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade','emp_title', 'issue_d',  
                          'zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 
                          'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 
                           'last_pymnt_amnt'], inplace = True)

In [3]:
print(loans_2007.head())

   loan_amnt        term int_rate  installment emp_length home_ownership  \
0     5000.0   36 months   10.65%       162.87  10+ years           RENT   
1     2500.0   60 months   15.27%        59.83   < 1 year           RENT   
2     2400.0   36 months   15.96%        84.33  10+ years           RENT   
3    10000.0   36 months   13.49%       339.31  10+ years           RENT   
4     3000.0   60 months   12.69%        67.79     1 year           RENT   

   annual_inc verification_status  loan_status pymnt_plan    ...      \
0     24000.0            Verified   Fully Paid          n    ...       
1     30000.0     Source Verified  Charged Off          n    ...       
2     12252.0        Not Verified   Fully Paid          n    ...       
3     49200.0     Source Verified   Fully Paid          n    ...       
4     80000.0     Source Verified      Current          n    ...       

  initial_list_status last_credit_pull_d collections_12_mths_ex_med  \
0                   f           Jun-201

The number of columns was reduced from 52 to 32 columns. For this task, our target column will be the **loan_status** column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower.

In [4]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the **Fully Paid** and **Charged Off** values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not.

We will treat the problem as a binary classification one. Let's remove all the loans that don't contain either **Fully Paid** and **Charged Off** as the loan's status and then transform the **Fully Paid** values to 1 for the positive case and the **Charged Off** values to 0 for the negative case.

In [5]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]

mapping_dict = {
    "loan_status": {
        "Fully Paid": 1,
        "Charged Off": 0
    }
}
loans_2007 = loans_2007.replace(mapping_dict)

Now, let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application.

In [6]:
drop_columns = []
for c in loans_2007.columns:
    non_null = loans_2007[c].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    if num_true_unique == 1:
        drop_columns.append(c)
        
loans_2007.drop(columns=drop_columns, inplace = True)
# print the dropped columns names
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


Therefore, we were able to remove 9 more columns since they only contained 1 unique value.

In [7]:
null_counts = loans_2007.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


In [9]:
object_columns_df = loans_2007.select_dtypes(include=['object'])

Now, we will remove all rows containing any missing values 

In [10]:
loans = loans_2007.dropna(axis=0)

Let's explore the unique value counts of the columnns that seem like they contain categorical values

In [12]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())
    print('--'*10)

RENT        17732
MORTGAGE    16435
OWN          2716
OTHER          96
Name: home_ownership, dtype: int64
--------------------
Not Verified       15586
Verified           11855
Source Verified     9538
Name: verification_status, dtype: int64
--------------------
10+ years    8426
< 1 year     4347
2 years      4219
3 years      3961
4 years      3308
5 years      3169
1 year       3084
6 years      2145
7 years      1692
8 years      1417
9 years      1211
Name: emp_length, dtype: int64
--------------------
 36 months    27538
 60 months     9441
Name: term, dtype: int64
--------------------
CA    6694
NY    3527
FL    2652
TX    2567
NJ    1743
IL    1435
PA    1433
VA    1314
GA    1298
MA    1234
OH    1132
MD     983
AZ     795
WA     765
CO     725
CT     701
NC     698
MI     669
MO     640
MN     574
NV     463
SC     446
OR     416
AL     413
LA     411
WI     410
KY     309
OK     284
KS     247
UT     243
AR     226
DC     206
RI     192
NM     174
WV     164
HI     161
NH  

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation). Lastly, the addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

In [13]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans.drop(columns=['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], inplace = True)
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')
loans = loans.replace(mapping_dict)

C:\Users\chaouki\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\chaouki\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\chaouki\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

Now, we encode the **home_ownership**, **verification_status**, **purpose**, and **term** columns as dummy variables so we can use them in our model.

In [14]:
columns = ['home_ownership', 'verification_status', 'purpose', 'term']
dummy_df = pd.get_dummies(loans[columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(columns, axis=1)

Upon inspecting the target column, we noticed that there's a class imbalance in our target column, loan_status. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Because of its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models